In [6]:
import cv2
import numpy as np
import os
from scipy.signal import savgol_filter

video_path = "C:\\Users\\chand\\Downloads\\SlowMoCric_5.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

ret, first_frame = cap.read()
if not ret:
    print("Error: Could not read first frame.")
    cap.release()
    exit()

bbox = cv2.selectROI("Select Ball (draw a box around the ball)", first_frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select Ball (draw a box around the ball)")

tracker = cv2.TrackerCSRT_create()
tracker.init(first_frame, bbox)

kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1e-4, 0], [0, 0, 0, 1e-4]], np.float32)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
orig_fps = cap.get(cv2.CAP_PROP_FPS)
output_fps = max(5, int(orig_fps) // 2)

output_video_path = "Ball_Trajectory_Overlay_sm.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, output_fps, (frame_width, frame_height))

positions = []
bat_hit = False

print("Tracking started. Press 'i' when the bat hits the ball to stop updating trajectory.")
print("Press 'q' to quit early.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if not bat_hit:
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            center = np.array([[x + w // 2], [y + h // 2]], np.float32)

            # Kalman filter correction
            kalman.correct(center)
            prediction = kalman.predict()
            pred_x, pred_y = int(prediction[0]), int(prediction[1])
            positions.append((pred_x, pred_y))
        else:
            pass  

    key = cv2.waitKey(1) & 0xFF
    if key == ord('i'):
        bat_hit = True  
    if key == ord('q'):
        break

    if len(positions) > 10:
        positions_arr = np.array(positions)
        window_size = min(len(positions), 11)
        if window_size % 2 == 0:
            window_size -= 1  # Ensure odd window size
        x_smooth = savgol_filter(positions_arr[:, 0], window_size, 2)
        y_smooth = savgol_filter(positions_arr[:, 1], window_size, 2)
        smooth_positions = list(zip(x_smooth.astype(int), y_smooth.astype(int)))
    else:
        smooth_positions = positions


    if len(smooth_positions) > 1:
        cv2.polylines(frame, [np.array(smooth_positions, np.int32)], False, (0, 255, 255), 4, cv2.LINE_AA)

    if positions:
        cv2.circle(frame, positions[-1], 8, (0, 0, 255), -1)

    cv2.putText(frame, "Press 'i' at bat impact | 'q' to quit", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow("Ball Trajectory Overlay", frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print("Output video saved as", output_video_path)


Tracking started. Press 'i' when the bat hits the ball to stop updating trajectory.
Press 'q' to quit early.


C:\Users\chand\AppData\Local\Temp\ipykernel_18600\3771028949.py:73: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(prediction[0]), int(prediction[1])


Output video saved as Ball_Trajectory_Overlay_sm.mp4
